## Data Loading

In [8]:
import pandas as pd

from collections import Counter
import time
import math

In [2]:
data = pd.read_csv("hien15k.csv")
data = data.drop(['Unnamed: 0'], axis=1)
# data.columns = ['hi-en','hi']
data

,hi-en,hi
0,suna,सुना
1,tha,था
2,dekha,देखा
3,na,न
4,samajha,समझा
...,...,...
15323,brownlo,ब्राउनलो
15324,roda,रोडा
15325,shymaleshwor,स्यामलेश्वर
15326,leonard,लियोनार्ड


In [3]:
data = data.drop_duplicates(subset=['hi-en'], ignore_index=True)
data = data.dropna()

In [4]:
len(data)

15327

In [9]:
sample = data['hi'][0]
print(sample)
print([c for c in sample])
print("".join([c for c in sample]))

सुना
['स', 'ु', 'न', 'ा']
सुना


## Cleaning

In [56]:
def clear_punk(text):
  punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}
  punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

  for p in punct_mapping:
    text = text.replace(p, punct_mapping[p])

  for p in punct:
    text = text.replace(p,' ')
  
  return text

In [57]:
def cleaner(data):
  hien = data['hi-en']
  hi = data['hi']

  hi = [clear_punk(text) for text in hi]
  hien = [clear_punk(text) for text in hien]

  data['hi-en'] = hien
  data['hi'] = hi

In [58]:
cleaner(data)

In [60]:
data

,hi-en,hi
0,suna,सुना
1,tha,था
2,dekha,देखा
3,na,न
4,samajha,समझा
...,...,...
15323,brownlo,ब्राउनलो
15324,roda,रोडा
15325,shymaleshwor,स्यामलेश्वर
15326,leonard,लियोनार्ड


# Modelling

In [61]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim

import torchtext as tt
from torchtext.vocab import Vocab

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

print(f"Torch Version: {torch.__version__}")
print(f"TorchText Version: {tt.__version__}")

Torch Version: 1.8.0+cu101
TorchText Version: 0.9.0


In [62]:
def tokenize(word):
  '''
      Tokenize the words
  '''
  return [c for c in word.lower()]

In [63]:
def build_vocab(data,min_freq ,tokenizer):
  counter = Counter()
  for word in data:
    counter.update(tokenizer(word))
  return Vocab(counter, min_freq=min_freq , specials=('<unk>', '<pad>', '<sos>', '<eos>'))

In [64]:
hi_char_vocab =  build_vocab(data['hi'], min_freq=1 ,tokenizer = tokenize)
hi_en_char_vocab = build_vocab(data['hi-en'],min_freq=75,tokenizer= tokenize)

In [65]:
print(len(hi_char_vocab))
print(len(hi_en_char_vocab))

109
31


In [66]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [67]:
def create_dataset(data, tokenizer, src_vocab, tar_vocab):
  src_raw_iter = iter(data['hi-en'])
  trg_raw_iter = iter(data['hi'])

  data = []

  for raw_src, raw_trg in zip(src_raw_iter, trg_raw_iter):
    src_tensor = torch.tensor([src_vocab[tok] for tok in tokenizer(raw_src)], dtype=torch.long)
    trg_tensor = torch.tensor([tar_vocab[tok] for tok in tokenizer(raw_trg)], dtype=torch.long)

    data.append((src_tensor,trg_tensor))

  return data

In [68]:
dataset = create_dataset(data, tokenize, hi_en_char_vocab, hi_char_vocab)

In [69]:
PAD_IDX = hi_char_vocab["<pad>"]
SOS_IDX = hi_char_vocab["<sos>"]
EOS_IDX = hi_char_vocab["<eos>"]

print(f"pad index: {PAD_IDX}")
print(f"sos index: {SOS_IDX}")
print(f"eos index: {EOS_IDX}")

pad index: 1
sos index: 2
eos index: 3


In [70]:
def batch_proc(data_batch):
  src_batch = []
  tar_batch = []
  
  for (src_item, tar_item) in data_batch:
    src_batch.append(torch.cat([torch.tensor([SOS_IDX]), src_item , torch.tensor([EOS_IDX])], dim=0))
    tar_batch.append(torch.cat([torch.tensor([SOS_IDX]), tar_item , torch.tensor([EOS_IDX])], dim=0))
  
  src_batch = pad_sequence(src_batch, padding_value= PAD_IDX)
  tar_batch = pad_sequence(tar_batch, padding_value= PAD_IDX)

  return src_batch, tar_batch

In [71]:
BATCH_SIZE = 64
train_iter = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn= batch_proc)

In [72]:
a = next(iter(train_iter))
print(a)

(tensor([[ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
        [17, 15, 10, 15, 23, 24, 17, 10, 23,  7, 15,  4, 10,  6, 22,  4,  4,  9,
         21, 11, 19,  4, 21, 17, 18, 25, 11, 14,  7, 15,  9,  5, 15,  6, 23, 21,
         21, 25, 11, 18, 17, 23, 18, 14, 15, 21,  4,  7,  4,  4, 14, 11, 17, 24,
          5, 21, 14, 17, 21, 11,  6, 14,  7, 16],
        [16,  4,  4, 13,  4,  5,  9,  4,  9,  4, 16, 24,  4,  4,  9, 12, 11,  8,
          4,  8, 16, 20,  4,  8,  8,  4,  7,  8,  4, 13,  8,  6,  4,  4,  4,  4,
         16,  4,  9,  4,  4,  4, 16, 16,  4,  4,  4, 16,  7,  6, 16,  4,  4,  4,
         11, 16,  9,  4, 16,  4,  4, 13, 16, 21],
        [15,  6, 22,  9,  9,  7, 13, 26,  4,  6,  7,  4, 14,  6, 16, 15,  9,  6,
         22, 22,  4,  8,  5,  5,  8, 11

## Transformer

In [73]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

In [74]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

In [75]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

In [76]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [77]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

In [78]:
class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

In [79]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

In [137]:
INPUT_DIM = len(hi_en_char_vocab)
OUTPUT_DIM = len(hi_char_vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

In [138]:
SRC_PAD_IDX = hi_en_char_vocab.stoi['<pad>']
TRG_PAD_IDX = hi_char_vocab.stoi['<pad>']

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [139]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,068,717 trainable parameters


In [140]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [141]:
model.apply(initialize_weights);

In [142]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [143]:
criterion = nn.CrossEntropyLoss()
length_loss = nn.MSELoss()

In [144]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch[0]
        trg = batch[1]
        
        
        src = src.permute(1,0).to(device)
        trg = trg.permute(1,0).to(device)

        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
        
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        loss = criterion(output, trg) + length_loss(torch.tensor(len(output)/50), torch.tensor(len(trg)/50))

        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [145]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch[0]
            trg = batch[1]

            src = src.permute(1,0).to(device)
            trg = trg.permute(1,0).to(device)

            output, _ = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [146]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [147]:
N_EPOCHS = 100
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, train_iter, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

    num = 1000
    word = data['hi-en'][num]
    expected = data['hi'][num]

    s = time.time()
    res, attention = transliterate_word(word, hi_en_char_vocab, hi_char_vocab, model, device)
    e = time.time()

    print(word)
    print(expected)
    print("".join(res))
    print(e-s)

Epoch: 01 | Time: 0m 10s
	Train Loss: 1.225 | Train PPL:   3.404
	 Val. Loss: 0.827 |  Val. PPL:   2.286
musibat
मुसीबत
बुस्ट्बिब्ब्ब्टम्ब्ब्ब्ब्ब्ब्ब्ट्ब्ब्ब्ट्ब्ब्स्ट्ब
0.16764044761657715
Epoch: 02 | Time: 0m 10s
	Train Loss: 0.833 | Train PPL:   2.301
	 Val. Loss: 0.626 |  Val. PPL:   1.871
musibat
मुसीबत
मस्बिमुस्ब्त्बरमस्स्बिम<eos>
0.08085083961486816
Epoch: 03 | Time: 0m 10s
	Train Loss: 0.632 | Train PPL:   1.882
	 Val. Loss: 0.472 |  Val. PPL:   1.604
musibat
मुसीबत
मुस्बित्स्बियमब्ट्टिबास्ट<eos>
0.0880594253540039
Epoch: 04 | Time: 0m 10s
	Train Loss: 0.528 | Train PPL:   1.696
	 Val. Loss: 0.412 |  Val. PPL:   1.510
musibat
मुसीबत
मुस्बित्ब्बिस्ब्ब्ब्ब्ब्बित्ब्ब्ब्ब्ब्बिस्ब्ब्ब्ब्
0.18646931648254395
Epoch: 05 | Time: 0m 10s
	Train Loss: 0.469 | Train PPL:   1.598
	 Val. Loss: 0.368 |  Val. PPL:   1.445
musibat
मुसीबत
मुस्बित्स्बाबाबेंट्स्बाबाबाबाबाबाबाबेंट्स्स्स्स्बा
0.16937470436096191
Epoch: 06 | Time: 0m 10s
	Train Loss: 0.434 | Train PPL:   1.544
	 Val. Loss: 0.344 |  V

KeyboardInterrupt: ignored

In [ ]:
def transliterate_word (word, src_vocab, trg_vocab, model, device, max_len = 50):
    
    model.eval()
        
    if isinstance(word, str):
        tokens = tokenize(word)
    else:
        tokens = [token.lower() for token in word]

    tokens = [src_vocab['<sos>']] + tokens + [src_vocab['<eos>']]
        
    src_indexes = [src_vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [trg_vocab.stoi['<sos>']]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_vocab.stoi['<eos>']:
            break

    trg_tokens = [trg_vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

In [128]:
for  num in range(10):
  # num = 10
  word = data['hi-en'][num]
  expected = data['hi'][num]

  s = time.time()
  res, attention = transliterate_word(word, hi_en_char_vocab, hi_char_vocab, model, device)
  e = time.time()

  print(word)
  print(expected)
  print("".join(res))
  print(e-s)
  print('\n')

suna
सुना
सुनअद्दुनादुल‍िनादुनादुनादुनादुनादुनाद्‍िक्‍सुनादु
0.21206927299499512


tha
था
थाड़ी<eos>
0.01846909523010254


dekha
देखा
देखड़ेखड़ेखड़ी<eos>
0.04222536087036133


na
न
नादप्‍दीनादाप्‍जन‍दन‍दन‍दन‍दन‍दन‍दन‍दन‍दन‍दन‍दाप्‍
0.17318964004516602


samajha
समझा
समझाअब्‍िसादाल्ता<eos>
0.06345057487487793


jana
जाना
जनाद्दनादीपीदादादनादादादादादादादादादादादादादादादना
0.1667799949645996


han
हां
हंचप्धंढपु<eos>
0.0372006893157959


kya
क्या
क्याद्दधधधधधधड़क्देश्दक्‍दक्‍ियादक्‍दक्‍दक्‍दक्‍दक्
0.17278361320495605


yahi
यही
यहिद्धियाधि<eos>
0.045709848403930664


vo
वो
वोवोडव्सवास्डवाबीवाल्डवोडवाज़ेस<eos>
0.10198736190795898




In [53]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_vocab, trg_vocab, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []

    src_data = data['hi-en']
    trg_data = data['hi']
    
    for src,trg in zip(src_data, trg_data):
        
        pred_trg, _ = transliterate_word(src, src_vocab, trg_vocab, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
    return bleu_score(pred_trgs, trgs)

In [ ]:
bleu_score = calculate_bleu(data, hi_en_char_vocab, hi_char_vocab, model, device)

print(f'BLEU score = {bleu_score*100:.2f}')

In [ ]:
def translate_sentence_vectorized(src_tensor, src_vocab, trg_vocab, model, device, max_len=50):
    assert isinstance(src_tensor, torch.Tensor)

    model.eval()
    src_mask = model.make_src_mask(src_tensor)

    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)
    # enc_src = [batch_sz, src_len, hid_dim]

    trg_indexes = [[trg_vocab.stoi['<sos>']] for _ in range(len(src_tensor))]
    # Even though some examples might have been completed by producing a <eos> token
    # we still need to feed them through the model because other are not yet finished
    # and all examples act as a batch. Once every single sentence prediction encounters
    # <eos> token, then we can stop predicting.
    translations_done = [0] * len(src_tensor)
    for i in range(max_len):
        trg_tensor = torch.LongTensor(trg_indexes).to(device)
        trg_mask = model.make_trg_mask(trg_tensor)
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        pred_tokens = output.argmax(2)[:,-1]
        for i, pred_token_i in enumerate(pred_tokens):
            trg_indexes[i].append(pred_token_i)
            if pred_token_i == trg_vocab.stoi['<eos>']:
                translations_done[i] = 1
        if all(translations_done):
            break

    # Iterate through each predicted example one by one;
    # Cut-off the portion including the after the <eos> token
    pred_sentences = []
    for trg_sentence in trg_indexes:
        pred_sentence = []
        for i in range(1, len(trg_sentence)):
            if trg_sentence[i] == trg_vocab.stoi['<eos>']:
                break
            pred_sentence.append(trg_vocab.itos[trg_sentence[i]])
        pred_sentences.append(pred_sentence)

    return pred_sentences, attention

In [ ]:
from torchtext.data.metrics import bleu_score

def calculate_bleu_alt(iterator, src_vocab, trg_vocab, model, device, max_len = 50):
    trgs = []
    pred_trgs = []
    with torch.no_grad():
        for batch in iterator:
            src = batch[0]
            trg = batch[1]

            src = src.permute(1,0).to(device)
            trg = trg.permute(1,0).to(device)

            _trgs = []
            for sentence in trg:
                tmp = []
                # Start from the first token which skips the <start> token
                for i in sentence[1:]:
                    # Targets are padded. So stop appending as soon as a padding or eos token is encountered
                    if i == trg_vocab.stoi['<eos>'] or i == trg_vocab.stoi['<pad>']:
                        break
                    tmp.append(trg_vocab.itos[i])
                _trgs.append([tmp])
            trgs += _trgs
            pred_trg, _ = translate_sentence_vectorized(src, src_vocab, trg_vocab, model, device)
            pred_trgs += pred_trg
    return bleu_score(pred_trgs, trgs)

In [ ]:
bleu_score = calculate_bleu_alt(train_iter, hi_en_char_vocab, hi_char_vocab, model, device)

print(f'BLEU score = {bleu_score*100:.2f}')